# Game Theory
## Payoff Matrix

In [1]:
import numpy as np
import math
import pandas as pd

In [14]:
class Game:
    def __init__(self, tab, actions, actions2=[], asymetrical=False):
        self.actions = actions
        m = np.array(tab, dtype=[("x", object), ("y", object)])
        if asymetrical:
            self.size = len(actions)
            self.size2 = len(actions2)
            self.actions2 = actions2
            self.scores = m.reshape(self.size, self.size2)
        else:
            self.size = int(math.sqrt(len(tab)))
            self.scores = m.reshape(self.size, self.size)
        self.asymetrical = asymetrical

    def getNash(self):
        max_x = np.matrix(self.scores["x"].max(0)).repeat(self.size, axis=0)
        bool_x = self.scores["x"] == max_x
        if self.asymetrical:
            max_y = (
                np.matrix(self.scores["y"].max(1))
                .transpose()
                .repeat(self.size2, axis=1)
            )
        else:
            max_y = (
                np.matrix(self.scores["y"].max(1)).transpose().repeat(self.size, axis=1)
            )
        bool_y = self.scores["y"] == max_y
        bool_x_y = bool_x & bool_y
        result = np.where(bool_x_y == True)
        listOfCoordinates = list(zip(result[0], result[1]))
        return listOfCoordinates

    def isPareto(self, t, s):
        return (
            True
            if (len(s) == 0)
            else (s[0][0] <= t[0] or s[0][1] <= t[1]) and self.isPareto(t, s[1:])
        )

    def getPareto(self):
        x = 0
        y = 0
        res = list()
        liste = self.scores.flatten()
        for s in liste:
            if x == self.size:
                x = 0
                y = y + 1
            if self.isPareto(s, liste):
                res.append((x, y))
            x = x + 1
        return res

    def getDominantStrategies(self, strict=True):
        dominatedLines = []
        dominatedColumns = []
        findDominated = True
        while (
            findDominated
            and (len(dominatedLines) != self.size - 1)
            and (len(dominatedColumns) != self.size - 1)
        ):
            findDominated = False
            # on regarde les lignes dominées
            for i in range(self.size - 1):
                line1 = self.scores["x"][i]
                line2 = self.scores["x"][i + 1]
                if self.compare(line1, line2, dominatedColumns, strict):
                    if i not in dominatedLines:
                        dominatedLines += [i]
                        findDominated = True
                if self.compare(line2, line1, dominatedColumns, strict):
                    if i + 1 not in dominatedLines:
                        dominatedLines += [i + 1]
                        findDominated = True
            # on regarde les colonnes dominées

            if self.asymetrical:
                lenY = self.size2
            else:
                lenY = self.size
            for i in range(lenY - 1):
                c1 = self.scores["y"].transpose()[i]
                c2 = self.scores["y"].transpose()[i + 1]
                if self.compare(c1, c2, dominatedLines, strict):
                    if i not in dominatedColumns:
                        dominatedColumns += [i]
                        findDominated = True
                if self.compare(c2, c1, dominatedLines, strict):
                    if i + 1 not in dominatedColumns:
                        dominatedColumns += [i + 1]
                        findDominated = True
        return self.result(dominatedLines, dominatedColumns)

    def compare(self, l1, l2, tab, strict):
        dominated = True
        for i in range(self.size):
            if strict:
                if (l1[i] < l2[i] and i not in tab) or i in tab:
                    dominated = dominated and True
                else:
                    dominated = dominated and False
            else:
                if (l1[i] <= l2[i] and i not in tab) or i in tab:
                    dominated = dominated and True
                else:
                    dominated = dominated and False
        return dominated

    def result(self, dominatedLines, dominatedColumns):
        x = list()
        y = list()
        res = list()
        tab = list()
        colums = list()
        rows = list()
        for i in range(self.size):
            if i not in dominatedLines:
                x.append(i)
                colums.append(self.actions[i])
        if self.asymetrical:
            lenY = self.size2
        else:
            lenY = self.size
        for i in range(lenY):
            if i not in dominatedColumns:
                y.append(i)
                if self.asymetrical:
                    rows.append(self.actions2[i])
                else:
                    rows.append(self.actions[i])
        for indX in x:
            for indY in y:
                res.append((indX, indY))
                tab.append(self.scores[indX][indY])
        print(res)
        return Game(tab, colums, rows, True)

    def prettyPrint(self):
        game = pd.DataFrame(np.nan, self.actions, self.actions, dtype=object)
        for i in range(self.size):
            for j in range(self.size):
                game.iat[i, j] = self.scores[i][j]
        print(game)

print("All is ok")

All is ok


In [3]:
# Some famous games

# battle of the sexes
gs=[(3,2),(1,1),(0,0),(2,3)]     

# chicken game
cg=[(-100,-100),(2,-2),(-2,2),(0,0)]     

# matching pennies
mp=[(1,-1),(-1,1),(-1,1),(1,-1)] 

# rock-paper-scissors
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)] 

# prisoner's dilemma
dp =[(3,3),(0,5),(5,0),(1,1)]
g = Game(dp,['C','D'])
g.prettyPrint()

        C       D
C  [3, 3]  [0, 5]
D  [5, 0]  [1, 1]


In [4]:
def getNash(self):
    max_x = np.matrix(self.scores["x"].max(0)).repeat(self.size, axis=0)
    bool_x = self.scores["x"] == max_x
    max_y = np.matrix(self.scores["y"].max(1)).transpose().repeat(self.size, axis=1)
    bool_y = self.scores["y"] == max_y
    bool_x_y = bool_x & bool_y
    result = np.where(bool_x_y == True)
    listOfCoordinates = list(zip(result[0], result[1]))
    return listOfCoordinates


print("All is ok")

All is ok


In [5]:
## Application in Battle of Sexes

gs = [(3, 2), (1, 1), (0, 0), (2, 3)]
g = Game(gs, ["Opera", "Stadium"])

# We recover the indexes of the Nash equilibrium(s)
listOfCoordinates = getNash(g)
print("The indexes of Nash's equilibrium : ")
print(listOfCoordinates)

# We print the moves corresponding to these equilibrium
print("The corresponding rounds : ")
for cor in listOfCoordinates:
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates:
    print(g.scores[cor[0]][cor[1]])

The indexes of Nash's equilibrium : 
[(0, 0), (1, 1)]
The corresponding rounds : 
Opera Opera
Stadium Stadium
The corresponding scores : 
(3, 2)
(2, 3)


In [6]:
# Find find Nash's equilibrium(s) (if any exist) for the rock-paper-scissors game 
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])


## Pareto Optimum

In [7]:
def isPareto(self, t, s):
    return (
        True
        if (len(s) == 0)
        else (s[0][0] <= t[0] or s[0][1] <= t[1]) and isPareto(self, t, s[1:])
    )


def getPareto(self):
    x = 0
    y = 0
    res = list()
    liste = self.scores.flatten()
    for s in liste:
        if x == self.size:
            x = 0
            y = y + 1
        if isPareto(self, s, liste):
            res.append((x, y))
        x = x + 1
    return res


print("All is ok")

All is ok


In [8]:
gs=[(3,2),(1,1),(0,0),(2,3)]
g = Game(gs,['Opera','Stadium'])

# We recover the indexes of the pareto optimum(s)
listOfCoordinates = getPareto(g)
print("The indexes of Pareto's optimums : ")
print(listOfCoordinates)

# We print the moves corresponding to these optimums
print("The corresponding rounds : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

The indexes of Pareto's optimums : 
[(0, 0), (1, 1)]
The corresponding rounds : 
Opera Opera
Stadium Stadium
The corresponding scores : 
(3, 2)
(2, 3)


In [9]:
# Find find Pareto's equilibrium(s) (if any exist) for the rock-paper-scissors game 

rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])

## Dominant Strategy Equilibrium

In [10]:
def getDominantStrategies(self, strict="True"):
    dominatedLines = []
    dominatedColumns = []
    findDominated = True
    while (
        findDominated
        and (len(dominatedLines) != self.size - 1)
        and (len(dominatedColumns) != self.size - 1)
    ):
        findDominated = False
        # We look at dominated lines
        for i in range(self.size - 1):
            line1 = self.scores["x"][i]
            line2 = self.scores["x"][i + 1]
            if compare(self, line1, line2, dominatedColumns, strict):
                if i not in dominatedLines:
                    dominatedLines += [i]
                    findDominated = True
            if compare(self, line2, line1, dominatedColumns, strict):
                if i + 1 not in dominatedLines:
                    dominatedLines += [i + 1]
                    findDominated = True
        # We look at dominated columns
        for i in range(self.size - 1):
            c1 = self.scores["y"].transpose()[i]
            c2 = self.scores["y"].transpose()[i + 1]
            if compare(self, c1, c2, dominatedLines, strict):
                if i not in dominatedColumns:
                    dominatedColumns += [i]
                    findDominated = True
            if compare(self, c2, c1, dominatedLines, strict):
                if i + 1 not in dominatedColumns:
                    dominatedColumns += [i + 1]
                    findDominated = True
    return result(self, dominatedLines, dominatedColumns)


def compare(self, l1, l2, tab, strict):
    dominated = True
    for i in range(self.size):
        if strict:
            if (l1[i] < l2[i] and i not in tab) or i in tab:
                dominated = dominated and True
            else:
                dominated = dominated and False
        else:
            if (l1[i] <= l2[i] and i not in tab) or i in tab:
                dominated = dominated and True
            else:
                dominated = dominated and False
    return dominated


def result(self, dominatedLines, dominatedColumns):
    x = list()
    y = list()
    res = list()

    for i in range(self.size):
        if i not in dominatedLines:
            x.append(i)
        if i not in dominatedColumns:
            y.append(i)

    for indX in x:
        for indY in y:
            res.append((indX, indY))

    return res


print("All is ok")

All is ok


In [11]:
gs=[(3,2),(1,1),(0,0),(2,3)]     
g = Game(gs,['Opera','Stadium'])


# We recover the indexes of the non-dominated strategy(ies):
print("Strongly dominated strategies method")
listOfCoordinates = getDominantStrategies(g)
print("Non-dominated strategies indexes : ")
print(listOfCoordinates)

# We print the moves corresponding to these strategies
print("The corresponding rounds : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

print(" ")

# We recover the indexes of the non-dominated strategy(ies)::
print("Weakly dominated strategies method")
listOfCoordinates = getDominantStrategies(g, strict="False")
print("Non-dominated strategies indexes : ")
print(listOfCoordinates)

# We print the moves corresponding to these strategies
print("The corresponding rounds : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

Strongly dominated strategies method
Non-dominated strategies indexes : 
[(0, 0), (0, 1), (1, 0), (1, 1)]
The corresponding rounds : 
Opera Opera
Opera Stadium
Stadium Opera
Stadium Stadium
The corresponding scores : 
(3, 2)
(1, 1)
(0, 0)
(2, 3)
 
Weakly dominated strategies method
Non-dominated strategies indexes : 
[(0, 0), (0, 1), (1, 0), (1, 1)]
The corresponding rounds : 
Opera Opera
Opera Stadium
Stadium Opera
Stadium Stadium
The corresponding scores : 
(3, 2)
(1, 1)
(0, 0)
(2, 3)


In [12]:
# Simplify the following abstract game. Is there an equilibrium in dominant strategies?
exo=[(3,6),(7,1),(4,8),(5,1),(8,2),(6,1),(6,0),(6,2),(3,2)]
g = Game(exo,['A','B','C'])

# the answer must be a single issue (B,B)->(8,2)
listOfCoordinates = getDominantStrategies(g)
print(listOfCoordinates)

[(1, 1)]


## Prisoner's Dilemma

In [15]:
g = Game(dp,['C','D'])


print("There is one Nash balance for the prisonner's dilemma")
listOfCoordinates = g.getNash()
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print(" ")

print("There are three Pareto optimas for the prisoner's dilemma")
listOfCoordinates = g.getPareto()
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print(" ")

print("The strictly dominant strategy for the prisoner's dilemma is the strategy where both players choose to Defect.")
listOfCoordinates = getDominantStrategies(g)
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print(" ")

There is one Nash balance for the prisonner's dilemma
D D
 
There are three Pareto optimas for the prisoner's dilemma
C C
D C
C D
 
The strictly dominant strategy for the prisoner's dilemma is the strategy where both players choose to Defect.
D D
 


## Random Game Matrix

In [16]:
x = np.random.randint(0, 5, (3,3))
y = np.random.randint(0, 5, (3,3))
couples = [(a,b) for a,b in zip(x.flatten(),y.flatten())]
g = Game(couples, None)
print(g.scores)


[[(3, 1) (1, 1) (3, 4)]
 [(4, 0) (2, 4) (4, 2)]
 [(2, 0) (3, 1) (2, 2)]]


In [17]:
# Find the Nash equilibrium, Pareto optimum and the dominated strategy for this game

import itertools;
import random;

def numberOfGames(valeurs, nbCoups):
    return len(valeurs)**((nbCoups**2)*2)

print("Number of games : ", numberOfGames([1,2],2))

def enumAllGames(valeurs, nbCoups):
    res = [q for q in itertools.product([p for p in itertools.product(list(valeurs), repeat=2)], repeat=nbCoups**2)]
    return [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]

n = enumAllGames([1,2],2)
print("Printing of 10 possible random sets in this set")
for i in range (10):
    print(random.choice(n))

Number of games :  256
Printing of 10 possible random sets in this set
[(2, 1), (1, 1), (1, 2), (1, 1)]
[(1, 1), (1, 2), (2, 1), (1, 2)]
[(2, 2), (2, 2), (1, 2), (2, 1)]
[(1, 1), (1, 1), (2, 1), (2, 2)]
[(1, 2), (1, 1), (1, 1), (2, 2)]
[(1, 1), (2, 1), (2, 1), (1, 2)]
[(1, 2), (1, 2), (2, 1), (1, 2)]
[(2, 2), (2, 1), (1, 1), (2, 1)]
[(1, 1), (1, 1), (1, 2), (1, 2)]
[(1, 2), (2, 1), (1, 2), (1, 1)]


In [18]:
def countNashEquilibria(valeurs, coups):
    results = [len(Game(i, None).getNash()) for i in enumAllGames(valeurs, coups)]
    return dict((i, results.count(i)) for i in set(results))


In [19]:
# How many 2-choices games built with (1, 2) have x Nash equilibria?
countNashEquilibria([1, 2], 2)

{0: 2, 1: 44, 2: 114, 3: 80, 4: 16}

In [20]:
nbCoups=2
res = [q for q in itertools.combinations([p for p in itertools.product([0,1,3,5], repeat=2)], nbCoups**2)]
games = [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]
print(str(len(games))+" games studied.")
r = []
for g in games:
    if ((sorted(Game(g,None).getPareto()) == sorted(Game(g,None).getNash())) and (len(set(g)) == len(g))):
        r.append(g)
r

1820 games studied.


[[(0, 0), (0, 1), (0, 3), (1, 5)],
 [(0, 0), (0, 1), (0, 3), (3, 5)],
 [(0, 0), (0, 1), (0, 3), (5, 5)],
 [(0, 0), (0, 1), (1, 0), (3, 3)],
 [(0, 0), (0, 1), (1, 0), (3, 5)],
 [(0, 0), (0, 1), (1, 0), (5, 3)],
 [(0, 0), (0, 1), (1, 0), (5, 5)],
 [(0, 0), (0, 1), (1, 1), (3, 3)],
 [(0, 0), (0, 1), (1, 1), (3, 5)],
 [(0, 0), (0, 1), (1, 1), (5, 3)],
 [(0, 0), (0, 1), (1, 1), (5, 5)],
 [(0, 0), (0, 1), (1, 3), (3, 5)],
 [(0, 0), (0, 1), (1, 3), (5, 5)],
 [(0, 0), (0, 1), (3, 0), (5, 3)],
 [(0, 0), (0, 1), (3, 0), (5, 5)],
 [(0, 0), (0, 1), (3, 1), (5, 3)],
 [(0, 0), (0, 1), (3, 1), (5, 5)],
 [(0, 0), (0, 1), (3, 3), (5, 5)],
 [(0, 0), (0, 3), (1, 0), (3, 5)],
 [(0, 0), (0, 3), (1, 0), (5, 5)],
 [(0, 0), (0, 3), (1, 1), (3, 5)],
 [(0, 0), (0, 3), (1, 1), (5, 5)],
 [(0, 0), (0, 3), (1, 3), (3, 5)],
 [(0, 0), (0, 3), (1, 3), (5, 5)],
 [(0, 0), (0, 3), (3, 0), (5, 5)],
 [(0, 0), (0, 3), (3, 1), (5, 5)],
 [(0, 0), (0, 3), (3, 3), (5, 5)],
 [(0, 0), (1, 0), (1, 1), (3, 3)],
 [(0, 0), (1, 0), (1